### Get LLM

In [ ]:
# TODO: notify if ollama server is running with model loaded
import subprocess, os
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI as LOpenAI
from dotenv import load_dotenv
load_dotenv('/workspace/repos/deep_fund/.env')

# model_name, ctx_len = "llama3.1:8b-instruct-q8_0", 128000
model_name, ctx_len = "qwen2.5:3b-instruct-q8_0", 128000
# ollama pull hf.co/mradermacher/SaulLM-54B-Instruct-i1-GGUF:Q6_K
# ollama pull hf.co/mradermacher/SaulLM-54B-Instruct-i1-GGUF:Q4_K_M

if "gpt-4o" in model_name:
    openai_key = os.getenv("OPENAI_API_KEY")
    os.environ["OPENAI_API_KEY"] = openai_key
    
    print(f"Using OpenAI {model_name}...")
    llm = LOpenAI(model=model_name, max_tokens=8000)
else:
    subout = subprocess.run(['ollama', 'list'], capture_output=True, text=True)
    if model_name in subout.stdout:
        print('Model loaded...')
    else:
        try: 
            print("Pulling Ollama model...")
            sub_out = subprocess.run(['ollama', 'pull', model_name], capture_output=True, text=True)
        except Exception as e: 
            print(f"Error pulling model: Is the Ollama server running?\n{e}")
    
    addtion_kwargs = {"max_new_tokens": 8000}
    system_prompt = "You create datasets for instruction fine-tuning for large language models."
    llm = Ollama(model=model_name, url="http://127.0.0.1:11434", context_window=ctx_len, model_type="chat", is_function_calling_model=False, 
                 request_timeout=4000.0, additional_kwargs=addtion_kwargs, system_prompt=system_prompt, keep_alive=0)
    print(llm.metadata)


https://github.com/yizhongw/self-instruct/blob/main/data/seed_tasks.jsonl

### Processing the input into "clean format"

In [1]:
text = """Why It Matters
One of the single most important concepts in the study of finance is the time value of money (TVM). This
concept puts forward the idea that a dollar received today is worth more than, and therefore preferable to, a
dollar received at some point in the future. The three primary reasons for this are that (1) money received now
can be saved or invested now and earn interest or a return, resulting in more money in the future; (2) any
promise of future payments of cash will always carry the risk of default; and (3) it is simple human nature for
people to prefer making their purchases of goods and services in the present rather than waiting to make
them at some future time.
For this reason, it is important to incentivize people to give up their present consumption patterns by offering
them greater value in the future. Based on the concept of TVM, it can be said that a dollar was worth more to
us, and thus carried more value, yesterday than it is to us today. It also then follows that a dollar in our
possession right now carries a greater value for us than a dollar we might receive tomorrow or at some other
point in the future.
The entire concept of the time value of money is particularly important because it allows savers and investors
to make better-informed decisions about what to do with their money. TVM can help a person understand
which option may be best based on the critical factors of overall risk, rates of interest, inflation, and return.
TVM can also be used to help a person understand how much money they’ll need to save in an interestbearing account in order to reach a desired financial goal, such as saving $50,000 in 10 years in an account that earns 4% compound interest each year. TVM is the key underlying principle of such important financial
analytical activities as retirement planning, corporate capital project evaluation, and even deciding on your
Time Value of Money I: Single Payment Value
7 • Why It Matters 191
own personal investments and bank accounts.
If the main concept behind the TVM is that a specific amount of money in hand now is worth more today than
that same amount of money will be worth tomorrow, you might think that a person would be better off
spending their money now rather than saving it for later use. However, we know that this is not always the
case. Sometimes it is simply a better idea to save your money. While inflation can have the effect of making a
dollar worth less tomorrow than it is worth today, the positive effect of compound interest works in favor of
savers and investors.
7.1 Now versus Later Concepts
Learning Outcomes
By the end of this section, you will be able to:
• Explain why time has an impact on the value of money.
• Explain the concepts of future value and present value.
• Explain why lump sum cash flow is the basis for all other cash flows.
How and Why the Passage of Time Affects the Value of Money
The concept of the time value of money (TVM) is predicated on the fact that it is possible to earn interest
income on cash that you decide to deposit in an investment or interest-bearing account. As times goes by,
interest is earned on amounts you have invested (present value), which effectively means that time will add
value (future value) to your savings. The longer the period of time you have your money invested, the more
interest income will accrue. Also, the higher the rate of interest your account or investment is earning, again,
the more your money will grow.
Understanding how to calculate values of money in the present and at different points in the future is a key
component of understanding the material presented in this chapter—and of making important personal
financial decisions in your future (see Figure 7.2).
Figure 7.2 The Time Value of Money It is better to be paid today, or you will lose out on the money you would have earned in
interest.
The Lump Sum Payment or Receipt
The most basic type of financial transaction involves a simple, one-time amount of cash, which can be either a
receipt (inflow) or a payment (outflow). Such a one-time transaction is typically referred to as a lump sum. A
lump sum consists of a one-off cash flow that occurs at any single point in time, present or future. Because it is
always possible to dissect more complex transactions into smaller parts, the lump sum cash flow is the basis
on which all other types of cash flow are treated."""

In [ ]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.postprocessor import LLMRerank
from rag_utils import create_llama_vector_index_rag
from llama_index.core.node_parser import SentenceSplitter

documents = None

# OLLAMA_HOST="http://127.0.0.1:11436" ollama start 
embed_model = OllamaEmbedding(
    # model_name="llama3.2:1b-instruct-q8_0", # other embed option "bge-m3"
    model_name="bge-m3", # other embed option "bge-m3"
    base_url="http://127.0.0.1:11436",
    ollama_additional_kwargs={"mirostat": 0},
)

policy_vector_index = create_llama_vector_index_rag(llm, 
                                                    embed_model, 
                                                    documents=documents,
                                                    persist_dir="/workspace/data/uvu_lit/vector_idx_poicy_manual",
                                                    vector_store_kwargs={"chunk_size": 500, "chunk_overlap": 30}
)

policy_query_engine = policy_vector_index.as_query_engine(
    llm=llm,
    similarity_top_k=10,
    node_postprocessors=[
        LLMRerank(
            llm=llm,
            choice_batch_size=5,
            top_n=3,
        )
    ],
    # see https://github.com/run-llama/llama_index/blob/f7c5ee5efbb6172e819f26d1705fcdf6114b11a3/llama-index-core/llama_index/core/response_synthesizers/type.py#L4
    response_mode="tree_summarize", # "accumulate", "compact_accumulate", "compact", "simple_summarize", "tree_summarize"
)

In [ ]:
Catastrophic forgetting
